In [1]:
import tkinter as tk
from tkinter import filedialog, Frame, Label, Button, ttk
from PIL import Image, ImageTk
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from keras.applications.resnet import preprocess_input
import tkinter.font as tkFont

In [2]:
# Load model
model = load_model("resnet_model_3.h5")

In [ ]:
THRESHOLD = 0.523
IMG_SIZE = (224, 224)


# Preprocess function
def preprocess_image_cv(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError(f"Cannot read image: {path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    return preprocess_input(img.astype("float32"))

In [4]:
# Upload and predict
def upload_and_predict():
    path = filedialog.askopenfilename(
        filetypes=[("X-Ray images", "*.png;*.jpg;*.jpeg"), ("All files", "*.*")]
    )
    if not path:
        return

    # Display selected image
    img = Image.open(path).resize((300, 300))
    tk_img = ImageTk.PhotoImage(img)
    img_label.config(image=tk_img)
    img_label.image = tk_img

    # Predict
    x = np.expand_dims(preprocess_image_cv(path), axis=0)
    prob = model.predict(x, verbose=0)[0, 0]

    # Update progress bar
    prog_bar["value"] = prob * 100
    root.update_idletasks()

    # Show badge
    if prob >= THRESHOLD:
        badge.config(text="🚨 Pneumonia", bg="#e57373")
    else:
        badge.config(text="✅ Normal", bg="#81c784")
    prob_label.config(text=f"Pneumonia Likelihood: {prob:.2%}")

In [ ]:
# Build GUI
root = tk.Tk()
root.title("Chest X‑Ray Pneumonia Detector")
root.configure(bg="#f4f7fc")  # Soft background
tkFont.Font(family="Open Sans", size=24, weight="bold")

# Header
title_font = tkFont.Font(family="Open Sans", size=26, weight="bold")
Label(
    root,
    text="🫁 Chest X‑Ray Pneumonia Detector",
    font=title_font,
    fg="#1565c0",
    bg="#f4f7fc",
).pack(pady=15)

# Upload button
btn_font = tkFont.Font(family="Open Sans", size=16)
Button(
    root,
    text="📁 Upload X‑Ray Image",
    command=upload_and_predict,
    font=btn_font,
    fg="white",
    bg="#42a5f5",
    relief="flat",
    padx=20,
    pady=10,
).pack(pady=10)

# Image preview
img_label = Label(root, bg="#f4f7fc")
img_label.pack(pady=10)

# Prediction frame
pred_frame = Frame(root, bg="#f4f7fc")
pred_frame.pack(pady=10)

# Confidence bar
prog_bar = ttk.Progressbar(
    pred_frame, orient="horizontal", length=300, mode="determinate"
)
prog_bar.pack(pady=5)

# Badge label
badge = Label(
    pred_frame,
    text="Awaiting image...",
    font=("Open Sans", 14),
    fg="white",
    bg="#90a4ae",
    padx=10,
    pady=5,
)
badge.pack(pady=5)

# Probability text
prob_label = Label(
    pred_frame,
    text="Pneumonia Likelihood: --",
    font=("Open Sans", 12),
    fg="#424242",
    bg="#f4f7fc",
)
prob_label.pack()

# Common signs panel
details = Frame(root, bg="#e3f2fd", bd=1, relief="solid")
details.pack(fill="x", padx=20, pady=15)
Label(
    details,
    text="Look for typical X‑Ray signs:",
    font=("Open Sans", 16, "bold"),
    bg="#e3f2fd",
    fg="#0d47a1",
).pack(anchor="w", padx=10, pady=5)
for sign in [
    "• Opacity or consolidation (Cloudy lung areas)",
    "• Air bronchograms (Visible airways)",
    "• Pleural effusion (Fluid around lungs)",
    "• Increased lung markings (More visible lung lines)",
]:
    Label(details, text=sign, font=("Open Sans", 14), bg="#e3f2fd", fg="#0d47a1").pack(
        anchor="w", padx=10
    )

root.mainloop()